In [1]:
import numpy as np
import pandas as pd
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 📌 **1. Veri Ön İşleme Fonksiyonu**
def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 🟢 1. Kategorik değişkenleri belirle (CatBoost kendi encode eder!)
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    # 🟢 2. Eksik değerleri doldur
    df.fillna(df.median(numeric_only=True), inplace=True)
    print("✅ Eksik değerler dolduruldu!")

    # Hedef değişkeni ayır
    y = df["rainfall"] if "rainfall" in df.columns else None
    X = df.drop(columns=["rainfall"]) if "rainfall" in df.columns else df

    print("✅ Veri hazırlandı!")
    
    return X, y, categorical_cols

# 📌 **2. CatBoost Model Eğitme Fonksiyonu**
def train_catboost(X, y, cat_features):
    """
    CatBoost Modeli ile eğitimi gerçekleştirir.
    """
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = CatBoostClassifier(
        iterations=1000,  # Epochs
        learning_rate=0.01,  # Daha yavaş öğrenme
        depth=8,  # Daha derin ağaçlar
        cat_features=cat_features,  # Kategorik sütunları otomatik işleme
        eval_metric="AUC",
        verbose=200,  # Her 200 iterasyonda bir log göster
        early_stopping_rounds=50
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

    y_pred = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_pred)

    print(f"✅ CatBoost Modeli Eğitildi! ROC AUC: {auc_score:.4f}")
    
    return model

# 📌 **3. Test Verisi Üzerinde Tahmin Yapma Fonksiyonu**
def predict_and_save(model, test_file, output_file, cat_features):
    X_test, _, _ = preprocess_data(test_file, is_train=False)

    predictions = model.predict_proba(X_test)[:, 1]
    
    # Test seti ID'sini al
    test_df = pd.read_csv(test_file)
    test_ids = test_df["id"]

    # Kaggle formatında CSV oluştur
    submission = pd.DataFrame({"id": test_ids, "rainfall": predictions})
    submission.to_csv(output_file, index=False)
    
    print(f"✅ Tahminler {output_file} dosyasına kaydedildi!")

# 📌 **4. Çalıştırma Kodu**
train_file = "train.csv"
test_file = "test.csv"
output_file = "catboost_submission.csv"

# 1️⃣ Veriyi işle
X_train, y_train, categorical_cols = preprocess_data(train_file, is_train=True)

# 2️⃣ Modeli eğit (CatBoost kullanıyoruz)
catboost_model = train_catboost(X_train, y_train, categorical_cols)

# 3️⃣ Test verisi üzerinde tahmin yap ve sonucu kaydet
predict_and_save(catboost_model, test_file, output_file, categorical_cols)


✅ Veri yüklendi: (2190, 13)
✅ Eksik değerler dolduruldu!
✅ Veri hazırlandı!
0:	test: 0.7916809	best: 0.7916809 (0)	total: 75.9ms	remaining: 1m 15s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8660203893
bestIteration = 41

Shrink model to first 42 iterations.
✅ CatBoost Modeli Eğitildi! ROC AUC: 0.8660
✅ Veri yüklendi: (730, 12)
✅ Eksik değerler dolduruldu!
✅ Veri hazırlandı!
✅ Tahminler catboost_submission.csv dosyasına kaydedildi!
